In [1]:
import yaml
import sys
import os
import numpy as np
import torch 
os.chdir('/home/jonfrey/PLR3')
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/lib'))

import loaders_v2
from loaders_v2 import GenericDataset
from rotations import * 
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from visu import plot_pcd, Visualizer
import copy
from scipy.spatial.transform import Rotation as R
from helper import re_quat
from PIL import Image, ImageDraw
from deep_im import LossAddS
import copy
#from deep_im import flow_to_trafo
from visu import Visualizer
import matplotlib.pyplot as plt
import k3d
#exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_ws_deepim_debug_natrix.yml'

env_cfg_path = '/home/jonfrey/PLR3/yaml/env/env_natrix_jonas.yml'
exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_evaluate_pose_estimation.yml'
h = 480
w = 640
import k3d

def load_from_file(p):
    if os.path.isfile(p):
        with open(p, 'r') as f:
            data = yaml.safe_load(f)
    else:
        raise ValueError
    return data

exp = load_from_file(exp_cfg_path)
env = load_from_file(env_cfg_path)

dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)

batch = dataset_train[13450][0] #bann 10450
points, choose, img, target, model_points, idx = batch[0:6]
depth_img, label_img, img_orig, cam = batch[6:10]
gt_rot_wxyz, gt_trans, unique_desig = batch[10:13]

real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render,h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask,  bb = batch[24:]



                

/home/jonfrey/PLR3/src/loaders_v2/dataset_ycb.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  init_trans = torch.normal(mean=torch.tensor(gt_trans), std=nt)
/home/jonfrey/PLR3/src/helper/bounding_box.py:203: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  masked_idx = (d != 0).nonzero()
/home/jonfrey/miniconda3/envs/track_latest/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the 

In [2]:
# Plotting Helper Functions (not needed)
class Drawer():
    def __init__(self):
        self.im_in_plot = 0
        self.data = []
        
    def disp_img_1d(self,img,hold=False, save=False, nr=0 , ret=False):
        self.data.append(img)
        p = '/home/jonfrey/Debug/Midterm2/'
        
        if not hold:
            fig = plt.figure(figsize=(6*2*len(self.data),7))
            ax = []
            for j,a in enumerate(self.data):
                ax.append( fig.add_subplot(1,len(self.data), j+1)  )
                
                ax[-1].get_xaxis().set_visible(False)
                ax[-1].get_yaxis().set_visible(False)
                pos = ax[-1].imshow( a, cmap='Reds' )
                
                fig.colorbar(pos, ax=ax[-1])
            plt.show()
            if save:
                fig.savefig(p+str(nr)+'.png', dpi=300)
                
            if ret: 
                if isinstance( self.data[0], torch.Tensor):
                    self.data[0] = self.data[0].numpy()
                    print('CONV')
                    
                print(self.data[0].shape)
                a = np.max(self.data[0])
                b = np.min(self.data[0])
                
                d = (self.data[0]-float(b))
                d = (d / ((float(a)-float(b))) )*255 
                d = np.uint8(d)
                img = Image.fromarray( d )
                return d
            self.data = []
            self.ax = []
            
Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]

def disp_alignment(depth, label, real):
    data = np.zeros((480,640,4), dtype=np.uint8)
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    t = real
    data[:,:,:3] = t.numpy() # red patch in upper left
    data_depth[:,:,:3] = t.numpy()
    data[:,:,3] = 70
    data[:,:,3][label==8] = 255
    
    min_val = torch.min( depth[depth!=0] )
    max_val = torch.max( depth[depth!=0] )
    val = torch.clamp( ((depth-min_val) // (max_val-min_val))*255, 0, 255)
    
    img = Image.fromarray(data, 'RGBA')
    display(img)

def plot_mask(mask):
    min_val = torch.min( mask )
    max_val = float( max(1,torch.max( mask )) )
    mask = torch.clamp( (mask-min_val) / (max_val-min_val)*255 ,0,255)
    
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    data_depth[:,:,3] = 255
    for i in range(480):
        for j in range(640):
            data_depth[i,j,:4] = np.array( cmaplist[ int(mask[i,j])] )*255
    data_depth[:,:,3] = 255
    data_depth[:,:,3][label==2] = 255
    img_depth = Image.fromarray(data_depth, 'RGBA')
    display(img_depth)

def plot_two_pcd_line(x, y, point_size=0.005, c1='g', c2='r'):
    if c1 == 'b':
        k = 245
    elif c1 == 'g':
        k = 25811000
    elif c1 == 'r':
        k = 11801000
    elif c1 == 'black':
        k = 2580
    else:
        k = 2580

    if c2 == 'b':
        k2 = 245
    elif c2 == 'g':
        k2 = 25811000
    elif c2 == 'r':
        k2 = 11801000
    elif c2 == 'black':
        k2 = 2580
    else:
        k2 = 2580

    col1 = np.ones(x.shape[0]) * k
    col2 = np.ones(y.shape[0]) * k2
    plot = k3d.plot(name='points')
    plt_points = k3d.points(x, col1.astype(np.uint32), point_size=point_size)
    plot += plt_points
    plt_points = k3d.points(y, col2.astype(np.uint32), point_size=point_size)
    plot += plt_points
    for i in range(min(100,x.shape[0]) ):
        plot += k3d.line([x[i],y[i]],shader='mesh', width=0.0005, color=0xff0000)
    
    plt_points.shader = '3d'
    plot.display()

def plot_hist(x,n_bins = 20):
    fig, axs = plt.subplots(1, 1, sharey=True, tight_layout=True)
    colors = ['lime']
    axs.hist(x, bins=n_bins, color=colors, label=colors)
    fig.show()
    return fig

In [3]:
# Helper functions 
def get_scale_for_erosion(ero_in):
    res = torch.sum ( ero_in, dim = (2,3))
    res[res < 5000] = 5
    res[res < 10000] = 10
    res[res < 30000] = 20
    res[res < 40000] = 25
    res[res < 50000] = 30
    res[res >= 50000] = 40
    return res

def eroision(t,size=3):
    "t: tensor shape BS, C, H,W"
    out_c = t.shape[1]
    kernel_tensor = torch.ones( (out_c,1,size,size) )
    print(size, kernel_tensor, t.shape)
    return torch.nn.functional.conv2d(t, kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size)

def eroision_batch(t,t_size):
    "t: tensor shape BS, C, H,W"
    "t_size: tensor shape BS"
    out_c = t.shape[1]
    for b in range( t.shape[0] ):
        size = int( t_size[b] )
        kernel_tensor = torch.ones( (out_c,1,size,size) )
        t[b] = (torch.nn.functional.conv2d(t[b][None], kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size))[0,:,:t.shape[2], :t.shape[3]]
    return t



In [4]:
# How to load a specific datapoint form the dataset (Not Needed)

# [('data/0003/001742',), tensor([8], dtype=torch.int32)]

desig = unique_desig[0]
desig = 'data/0003/001742'

_p_ycb = "/media/scratch1/jonfrey/datasets/YCB_Video_Dataset"
depth = np.array(Image.open(
    '{0}/{1}-depth.png'.format(_p_ycb, desig)))
depth.shape

label = np.array(Image.open(
    '{0}/{1}-label.png'.format(_p_ycb, desig)))
img = np.array(Image.open(
    '{0}/{1}-color.png'.format(_p_ycb, desig)))
batch = dataset_train._backend.getElement( desig, 8)
batch = batch #bann 10450   
model_points = batch[4]
idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
real_img_original = batch[8]
cam = batch[9]
real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask, bb = batch[24:]


In [5]:
Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]
import time

In [6]:
# set loader to BS 1
exp['loader']['batch_size'] = 1
exp['loader']['pin_memory'] = False
exp['loader']['shuffle'] = True


exp['d_train']["output_cfg"]['overfitting_nr_idx'] = -1
dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)


# get test and train dataset
dataloader_train = torch.utils.data.DataLoader(dataset_train,
                                                       **exp['loader'])
exp['d_test']["output_cfg"]['overfitting_nr_idx'] = -1
exp['d_test']["flow_cfg"]['sub'] = 1
exp['d_test']["flow_cfg"]['min_matches'] = 50
exp['d_test']["flow_cfg"]['max_matches'] = 3000
exp['d_test']["flow_cfg"]['max_iterations'] = 20000
exp['d_test']["flow_cfg"]['max_iterations'] = 20000


dataset_test = GenericDataset(
    cfg_d=exp['d_test'],
    cfg_env=env)
dataloader_test = torch.utils.data.DataLoader(dataset_test,
                                                       **exp['loader'])

# get Loss function
criterion_adds = LossAddS(sym_list=exp['d_train']['obj_list_sym'])





In [7]:
# Numpy ICP implementation from https://github.com/ClayFlannigan/icp/blob/master/icp.py

import numpy as np
from sklearn.neighbors import NearestNeighbors


def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

In [8]:
idxmax = -1 
def get_H(pcd):
    pcd_ret = torch.ones( (pcd.shape[0],pcd.shape[1]+1),device=pcd.device, dtype=pcd.dtype )
    pcd_ret[:,:3] = pcd
    return pcd_ret

def eval_T(P_real_in_center, P_ren_in_center, T_res):
        """
        NR,3
        NR,3 
        4,4
        """
        P_ren_H = get_H( P_ren_in_center )
        P_ren_trafo =  (P_ren_H @ T_res.T)[:,:3]
        L2_dis_post = torch.mean( torch.norm( P_real_in_center-P_ren_trafo, dim=1 ) )
        L2_dis_pre = torch.mean( torch.norm( P_real_in_center-P_ren_in_center, dim=1 ) )
        return L2_dis_post, L2_dis_pre  

In [9]:
import torch
import numpy as np
import copy
from helper import anal_tensor

def solve_transform(keypoints, gt_keypoints):
    """
    keypoints: N x K x 3
    gt_keypoints: K x 3
    return: N x 4 x 4 transformation matrix
    """
    try:
        keypoints = keypoints.clone()
        gt_keypoints = gt_keypoints.clone()
        N, K, _ = keypoints.shape
        center = keypoints.mean(dim=1)
        gt_center = gt_keypoints.mean(dim=0)
        keypoints -= center[:, None, :]
        gt_keypoints -= gt_center[None]
        matrix = keypoints.transpose(2, 1) @ gt_keypoints[None]
        U, S, V = torch.svd(matrix)
        
        Vt = V.transpose(2, 1)
        Ut = U.transpose(2, 1)

        d = (V @ Ut).det()
        I = torch.eye(3, 3, dtype=gt_center.dtype, device= keypoints.device)[None].repeat(N, 1, 1)
        I[:, 2, 2] = d.clone()

        R = U @ I @ Vt
        T = torch.zeros(N, 4, 4, dtype=gt_center.dtype, device= keypoints.device)
        T[:, 0:3, 0:3] = R
        T[:, 0:3, 3] = center[None] - (R @ gt_center[None :, None])[:, :, 0]
        T[:, 3, 3] = 1.0

        return T
    except RuntimeError as error:
        import ipdb; ipdb.set_trace()
        print("Something went wrong")

# costume implementation 
def solve_transform2(A,B):
    if A.shape[0] > B.shape[0]:
        x=torch.arange(A.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:B.shape[0]]
        A = torch.index_select(A, 0, out)
    if A.shape[0] < B.shape[0]:
        x=torch.arange(B.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:A.shape[0]]
        B = torch.index_select(B, 0, out)

    #A = torch.choice(A,B.shape[0])

    assert A.shape == B.shape

    m = A.shape[1]
    centroid_A = torch.mean(A, dim=0)
    centroid_B = torch.mean(B, dim=0)

    AA = (A - centroid_A)
    BB = (B - centroid_B)
    H = AA.transpose(0,1) @ BB
    U, S, Vt = torch.svd(H)
    R = Vt @ U.transpose(0,1)
    if torch.det(R) < 0:
        Vt[m-1,:] *= -1
        R = Vt.transpose(0,1) @ U.transpose(0,1)

    # translation
    t = centroid_B - (R @ centroid_A)
    # homogeneous transformation
    T = torch.eye(m+1, device= A.device)
    T[:m, :m] = R
    T[:m, m] = t
    return T


# NR = 1000
# DIM = 3
# A = torch.ones( (NR,DIM), dtype= torch.float32)
# B = A*1.7223
# T = solve_transform2(A,B)
# print(T)


# A_hom = get_H(A)
# A_hom2 = A_hom @ T.T
# print(A_hom2[:,:3])

def filter_pcd_given_depthmap(pcd, depth, scal= 10000):
    """
    pcd = Nx3 troch.float32
    depth = N torch.float32

    return N torch.bool
    """
    m1 = (depth/scal) > 0.2
    #print( "Thorwn away values", (depth/scal) < 0.2 )
    return m1

    val_d = depth[ m1 ]
    mean = torch.mean(val_d)
    new_d = depth - mean
    tol = 0.5
    m2 = torch.abs( new_d/scal ) < tol 
    return m1 * m2
    
def filter_pcd( pcd, tol = 0.6):
    """
    input:
        pcd : Nx3 torch.float32
    returns:
        mask : NX3 torch.bool 
    """
    m = torch.mean(pcd, dim = 0)
    comp = m[None,:].repeat(pcd.shape[0],1) + tol
    mean_free = pcd-m[None,:].repeat(comp.shape[0],1)
    mask = torch.norm( mean_free,  dim= 1) > tol
    #print(f"filter_pcd PRE: {pcd.shape}, POST: {float(torch.sum(mask[:,None].repeat(1,3) == False ))/3.0}")
    return mask[:,None].repeat(1,3) == False

def filter_pcd_cor(pcd1, pcd2, max_mean_deviation=0.2):
    
    dif = torch.norm( pcd1-pcd2 , dim= 1)
    mean = torch.mean(dif, dim = 0)
    mean_free = torch.abs(dif-mean)
    #print(f"filter_pcd_cor PRE: {pcd1.shape[0]}, POST: {torch.sum(mean_free < max_mean_deviation)}")
    return mean_free < max_mean_deviation
    
def flow_to_trafo(*args, **kwargs):
    """
    input:
      real_br: torch.tensor torch.Size([2])
      real_tl: torch.tensor torch.Size([2])
      ren_br: torch.tensor torch.Size([2])
      ren_tl: torch.tensor torch.Size([2])
      flow_mask: torch.Size([480, 640])
      u_map: torch.Size([480, 640])
      v_map: torch.Size([480, 640])
      K_real: torch.Size([3, 3])
      K_ren: torch.Size([3, 3])
      real_d: torch.Size([480, 640]) 
      render_d: torch.Size([480, 640])
      h_real: torch.Size([4, 4])
      h_render: torch.Size([4, 4])
    output:
      P_real_in_center: torch.Size([N, 3])
      P_ren_in_center: torch.Size([N, 3]) 
      P_real_trafo: torch.Size([N, 3])
      T_res: torch.Size([4, 4])
      
      The output rotation T_res is defined in the Camera coordinate frame. 
      Therfore premultiply the T_Res with h_render to get the new h_real_new !!!
    """
    for k in kwargs.keys():
        pass
        #print(f"Variable: {k}, Type {type(kwargs[k])}, Dtype{kwargs[k].dtype}, Shape{kwargs[k].shape}")
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    plot_pcd = kwargs.get('plot_pcd',False)
  
    # Grid for upsampled real
    a = float(real_br[0]-real_tl[0])/480*1.0000001
    b = float(real_br[1]-real_tl[1])/640*1.0000001
    grid_real_h = torch.arange(int(real_tl[0]) ,int(real_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.arange(int(real_tl[1]) ,int(real_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)


    # Grid for upsampled ren
    a = float(ren_br[0]-ren_tl[0])/480*1.0000001
    b = float(ren_br[1]-ren_tl[1])/640*1.0000001
    c = 0
    
    grid_ren_h = torch.arange(int(ren_tl[0]) ,int(ren_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.arange(int(ren_tl[1]) ,int(ren_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)
    # Calculate valid depth map for rendered image
    render_d_ind_h = torch.arange(0 ,480 , 1, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.arange(0 ,640 , 1, device=u_map.device)[None,:].repeat(480,1)

    render_d_ind_h = torch.clamp(torch.round((render_d_ind_h - u_map).type(torch.float32)) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp(torch.round((render_d_ind_w - v_map).type(torch.float32)),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions
    ren_d_masked  = render_d.flatten()[index]
    
    # Project depth map to the pointcloud real
    cam_scale = 10000

    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_real.type(torch.float32)).type(u_map.dtype)
    P_real = K_inv @ real_pixels.T
    P_real = P_real * real_d[flow_mask] / cam_scale
    P_real = P_real.T
    
    # Project depth map to the pointcloud render
    K_ren_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
    ren_pixels = torch.stack( [grid_ren_w[flow_mask] - v_map[flow_mask], 
                            grid_ren_h[flow_mask] - u_map[flow_mask],
                            torch.ones(grid_ren_h.shape, device = u_map.device,  dtype= u_map.dtype )[flow_mask]], 
                            dim=1 ).type(u_map.dtype)
    P_ren = K_ren_inv @ ren_pixels.T
    P_ren = P_ren * ren_d_masked / cam_scale
    P_ren = P_ren.T

    # Filter the pointclouds given the depthmap
    m_ren_depth = filter_pcd_given_depthmap(P_ren, render_d[flow_mask])
    m_real_depth = filter_pcd_given_depthmap(P_real, real_d[flow_mask])
    m_total =  m_ren_depth * m_real_depth
    
    min_points = 20
    if torch.sum(m_total) < min_points:
        print(f'Violation filter pcd_given_depthmap: P_in: {P_ren.shape[0]} P_out: {torch.sum(m_total)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_ren = P_ren[m_total] 
    P_real = P_real[m_total]
    # anal_tensor(  P_ren, 'P_ren m_total masked')

    # Do not transfrom to center coordinate system
    P_real_in_center = P_real                      
    P_ren_in_center = P_ren 
    
    m_real = filter_pcd( P_real_in_center )
    m_ren = filter_pcd( P_ren_in_center )
    m_tot = m_real * m_ren
    if torch.sum(m_tot) < min_points:
        print(f'Violation filter_pcd: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_tot)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_tot[:,0]]
    P_ren_in_center = P_ren_in_center[m_tot[:,0]]
  
    # Max mean deviation
    m_new = filter_pcd_cor(P_real_in_center, P_ren_in_center)
    
    if torch.sum(m_new) < min_points:
        print(f'Violation filter_pcd_cor: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_new)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_new]
    P_ren_in_center = P_ren_in_center[m_new]

    # random shuffel
    pts_trafo = min( P_real_in_center.shape[0], 1000 )
    idx = torch.randperm( P_real_in_center.shape[0] )[0:pts_trafo]
    P_real_in_center = P_real_in_center[idx]
    P_ren_in_center = P_ren_in_center[idx]

    T_res = solve_transform( P_real_in_center[None].type(torch.float64 ) , P_ren_in_center.type(torch.float64 ) ).type(u_map.dtype )
    
    # Transform the real points according to calculated transformation
    P_hr = torch.ones( (P_real_in_center.shape[0],4 ) , device=u_map.device, dtype= u_map.dtype)
    P_hr[:,:3] = P_real_in_center
    P_real_trafo = (torch.inverse( T_res[0].type(torch.float32) ).type(u_map.dtype ) @ copy.deepcopy(P_hr).T).T [:,:3]

    return True, P_real_in_center, P_ren_in_center, P_real_trafo, T_res[0]



In [33]:
import cv2 
cv2.__version__
from scipy.spatial.transform import Rotation as R


def flow_to_trafo_pnp(*args, **kwargs):
    """
    input:
      real_br: torch.tensor torch.Size([2])
      real_tl: torch.tensor torch.Size([2])
      ren_br: torch.tensor torch.Size([2])
      ren_tl: torch.tensor torch.Size([2])
      flow_mask: torch.Size([480, 640])
      u_map: torch.Size([480, 640])
      v_map: torch.Size([480, 640])
      K_real: torch.Size([3, 3])
      K_ren: torch.Size([3, 3])
      real_d: torch.Size([480, 640]) 
      render_d: torch.Size([480, 640])
      h_real: torch.Size([4, 4])
      h_render: torch.Size([4, 4])
    output:
      P_real_in_center: torch.Size([N, 3])
      P_ren_in_center: torch.Size([N, 3]) 
      P_real_trafo: torch.Size([N, 3])
      T_res: torch.Size([4, 4])
      
      The output rotation T_res is defined in the Camera coordinate frame. 
      Therfore premultiply the T_Res with h_render to get the new h_real_new !!!
    """
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    h_real_est = kwargs['h_real_est']
    

    # Grid for upsampled real
    a = float(real_br[0]-real_tl[0])/480*1.0000001
    b = float(real_br[1]-real_tl[1])/640*1.0000001
    grid_real_h = torch.arange(int(real_tl[0]) ,int(real_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.arange(int(real_tl[1]) ,int(real_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)


    # Grid for upsampled ren
    a = float(ren_br[0]-ren_tl[0])/480*1.0000001
    b = float(ren_br[1]-ren_tl[1])/640*1.0000001
    c = 0
    
    grid_ren_h = torch.arange(int(ren_tl[0]) ,int(ren_br[0]) , a, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.arange(int(ren_tl[1]) ,int(ren_br[1]) , b, device=u_map.device)[None,:].repeat(480,1)
    # Calculate valid depth map for rendered image
    render_d_ind_h = torch.arange(0 ,480 , 1, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.arange(0 ,640 , 1, device=u_map.device)[None,:].repeat(480,1)

    render_d_ind_h = torch.clamp(torch.round((render_d_ind_h - u_map).type(torch.float32)) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp(torch.round((render_d_ind_w - v_map).type(torch.float32)),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions
    ren_d_masked  = render_d.flatten()[index]
    
    # Project depth map to the pointcloud real
    cam_scale = 10000

    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_real.type(torch.float32)).type(u_map.dtype)
    P_real = K_inv @ real_pixels.T
    P_real = P_real * real_d[flow_mask] / cam_scale
    P_real = P_real.T
    
    # Project depth map to the pointcloud render
    K_ren_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
    ren_pixels = torch.stack( [grid_ren_w[flow_mask] - v_map[flow_mask], 
                            grid_ren_h[flow_mask] - u_map[flow_mask],
                            torch.ones(grid_ren_h.shape, device = u_map.device,  dtype= u_map.dtype )[flow_mask]], 
                            dim=1 ).type(u_map.dtype)
    P_ren = K_ren_inv @ ren_pixels.T
    P_ren = P_ren * ren_d_masked / cam_scale
    P_ren = P_ren.T

    # Filter the pointclouds given the depthmap
    m_ren_depth = filter_pcd_given_depthmap(P_ren, render_d[flow_mask])
    m_real_depth = filter_pcd_given_depthmap(P_real, real_d[flow_mask])
    m_total =  m_ren_depth * m_real_depth
    
    min_points = 20
    if torch.sum(m_total) < min_points:
        print(f'Violation filter pcd_given_depthmap: P_in: {P_ren.shape[0]} P_out: {torch.sum(m_total)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)
    
    real_pixels_fil = real_pixels[m_total]
    P_ren = P_ren[m_total] 
    P_real = P_real[m_total]
    # anal_tensor(  P_ren, 'P_ren m_total masked')

    # Do not transfrom to center coordinate system
    P_real_in_center = P_real                      
    P_ren_in_center = P_ren 
    
    m_real = filter_pcd( P_real_in_center )
    m_ren = filter_pcd( P_ren_in_center )
    m_tot = m_real * m_ren
    if torch.sum(m_tot) < min_points:
        print(f'Violation filter_pcd: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_tot)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_tot[:,0]]
    P_ren_in_center = P_ren_in_center[m_tot[:,0]]
    real_pixels_fil = real_pixels_fil[m_tot[:,0]]
        
    # Max mean deviation
    m_new = filter_pcd_cor(P_real_in_center, P_ren_in_center)
    
    if torch.sum(m_new) < min_points:
        print(f'Violation filter_pcd_cor: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_new)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_new]
    P_ren_in_center = P_ren_in_center[m_new]
    real_pixels_fil = real_pixels_fil[m_new]
    
    # random shuffel
    pts_trafo = P_real_in_center.shape[0]
    idx = torch.randperm( P_real_in_center.shape[0] )[0:pts_trafo]
    P_real_in_center = P_real_in_center[idx]
    P_ren_in_center = P_ren_in_center[idx]
    real_pixels_fil = real_pixels_fil[idx]
    
    
    # The same filtering is performed for the PnP as for the Pointcloud matching approach! 
    
    # Transform the known rendered point corrspondences to the origin frame. 
    P_ren_h = get_H(P_ren_in_center)
    P_ren_h_trafo = P_ren_h @ torch.inverse(h_render).T

    # define PnP inputs
    objectPoints = P_ren_h_trafo[:,:3].numpy() #P_ren.numpy()    
    imagePoints = real_pixels_fil[:,:2].numpy()
    dist = np.array( [[0.0,0.0,0.0,0.0]] )
    div = 1 # int(objectPoints.shape[0]/1000)
    
    # set current guess as the inital estimate
    rvec = R.from_matrix(h_real_est[:3,:3]).as_rotvec().astype(np.float32)
    tvec = h_real_est[:3,3].numpy().astype(np.float32)
    
    # calculate PnP between the pixels coordinates in the real image and the corrosponding points in the origin frame
    rvec, tvec = cv2.solvePnPRefineLM(objectPoints[::div], \
        imagePoints[::div], K_real.numpy(), dist, rvec, tvec)
    
    #retval, rvec, tvec, inliers = cv2.solvePnPRansac(objectPoints[::div], \
    #    imagePoints[::div], K_real.numpy(), dist, reprojectionError = 1, iterationsCount= 1000,confidence= 0.9999999)
    # print(inliers)
    
    # get homogenous output transformation
    rot = R.from_rotvec(rvec[:])
    h = np.eye(4)
    h[:3,:3] = rot.as_matrix()
    h[:3,3] = tvec.T

    return torch.from_numpy( h).type(u_map.dtype) 
    
# flow_to_trafo_pnp(
#     real_br = real_br[b].clone(),
#     real_tl = (real_tl[b]).clone(), 
#     ren_br = (ren_br[b]).clone(), 
#     ren_tl = (ren_tl[b]).clone(),
#     flow_mask = (flow_mask[b]).clone(), 
#     u_map = (u_map[b].type( typ )).clone(),
#     v_map = (v_map[b].type( typ )).clone(), 
#     K_real = (K_real.type( typ )).clone(),
#     K_ren = (K_ren.type( typ )).clone(),
#     real_d = (real_d[b].type( typ )).clone(),
#     render_d = (render_d[b].type( typ )).clone(),
#     h_real = (h_real_est.type( typ )).clone(), 
#     h_render = (h_render[b].type( typ )).clone(),
#     h_real_est = h_real_est)

tensor([[ 0.9763, -0.0734, -0.2035, -0.0899],
        [-0.2107, -0.5348, -0.8183,  0.0053],
        [-0.0487,  0.8418, -0.5376,  0.9096],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

In [30]:
from visu import plot_two_pcd
import itertools
# idxmax can be set to != -1 to load specific sample

idxmax = -1 
device = 'cpu'
desig_ls= []
adds_gt_ls = []
adds_init_ls = []
adds_erode_ls = []
adds_n1_ls = []
adds_n2_ls = []

adds_pnp_gt_ls = []
adds_pnp_ero_ls = []
visualizer = Visualizer('/home/jonfrey/Debug', None)
print('START')

K_ren = torch.tensor( dataset_train._backend.get_camera('data_syn/0019', K=True), device=device ) 
max_iter = 1000

if idxmax != -1:
    print(idxmax, "is not equal to -1 load batch")
    batch = next(itertools.islice(dataloader_train, idxmax, None))[0]

for j,ba in enumerate(dataloader_test):
    if j > max_iter-1:
        break
    if j % 10 == 0:
        print(f"Processed {j}/{max_iter}")
    if idxmax == -1:  
        batch = ba[0]
    # use first sample in batch 
    b = 0
    
    model_points = batch[4]
    idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
    label = batch[7]
    real_img_original = batch[8]
    cam = batch[9]
    gt_rot_wxyz, gt_trans, unique_desig = batch[10:13] # unique_desig[1] contains the idx starting at 1 for the first object 
    bs = model_points.shape[0]
    if batch[13] is False:
        print('Continue')
        continue
    real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
    pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
    u_map, v_map, flow_mask, bb = batch[24:]
    real_tl, real_br, ren_tl, ren_br = bb 
    
    data = torch.cat([real_img, render_img], dim=1)
    uv_gt = torch.stack( [u_map, v_map], dim=3 ).permute(0,3,1,2)
    
    
    ero_in = (gt_label_cropped ==  unique_desig[1])[:,None,:,:].type(torch.float32) # BS,C,H,W
    t_size = get_scale_for_erosion(ero_in)
    ero_out = eroision_batch(ero_in,t_size)
    
    # get camera
    K_real = torch.tensor( [[cam[b,2],0,cam[b,0]],[b,cam[b,3],cam[b,1]],[0,0,1]], device=device )
    
    #get inital estimate of the poistion given by the dataloader
    h_real_est = torch.eye(4,device=device)
    h_real_est[:3,:3] = quat_to_rot(pred_rot_wxyz[b][None,:], conv='wxyz', device=device)
    h_real_est[:3,3] = torch.tensor( pred_trans[b].clone().detach() ,device=device )
    
    ### GT Estimate ###
    typ = u_map.dtype
    fmt = flow_mask.dtype
    
     ### GT Label ###
    suc, P_real_in_center, P_ren_in_center, P_real_trafo, gt_T_res = flow_to_trafo(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real_est.type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone())
    gt_h_est =  gt_T_res @ h_render[0].type(typ)
    if not suc:
        continue
    pnp_h_gt = flow_to_trafo_pnp(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real_est.type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone(),
        h_real_est = h_real_est)
    

    if not suc:
        continue
    ### Erode Label ###
    flow_mask_eroded  = (flow_mask * ero_out.type(torch.float32)).type(fmt)[:,0]
    suc,_,_,_, ero_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = copy.deepcopy((u_map[b]).type( typ )), 
        v_map = copy.deepcopy((v_map[b]).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    ero_h_est =  ero_T_res @ h_render[0].type(typ) # set rotation
    if not suc:
        continue
    pnp_h_ero = flow_to_trafo_pnp(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask_eroded[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real_est.type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone(),
        h_real_est = h_real_est)
    
    
    ### N1 ###
    lvl = 20
    n1 = torch.rand(v_map[b].shape, dtype = typ)
    n2 = torch.rand(v_map[b].shape, dtype = typ)
    u_map_clone = (u_map[b]).type( typ ).clone() + (n1 - 0.5) * 2 * lvl
    v_map_clone = (v_map[b]).type( typ ).clone() + (n2 - 0.5) * 2 * lvl
    
    suc,_,_,_, n1_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n1_h_est =  n1_T_res @ h_render[0].type(typ) # set rotation
    if not suc:
        continue
    ### N2 ###
    lvl = 20
    n1 = float(torch.rand((1), dtype = typ ))
    n2 = float(torch.rand((1), dtype = typ ))
    u_map_clone = (u_map[b]).type( typ ).clone() + float( (n1 - 0.5) * 2 * lvl )
    v_map_clone = (v_map[b]).type( typ ).clone() + float( (n2 - 0.5) * 2 * lvl )
    
    
    suc,_,_,_, n2_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n2_h_est =  n2_T_res @ h_render[0].type(typ) # set rotation
    
    
    if not suc:
        continue
    mask = (flow_mask == True)

    # Target Model-points Transformed
    p = model_points.shape[1]
    target = torch.bmm( model_points, torch.transpose(h_real[:,:3,:3], 1,2 ) ) + h_real[:,:3,3][:,None,:].repeat(1,p,1)

    # Compute ADD-S
    adds_res_gt_flow = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = gt_h_est[None].type( target.dtype) )
    adds_res_gt_flow_eroded = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = ero_h_est[None].type( target.dtype) )
    adds_res_n1 = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = n1_h_est[None].type( target.dtype) )
    adds_res_n2 = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = n2_h_est[None].type( target.dtype) )
    
    adds_res_pnp_gt = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = pnp_h_gt[None].type( target.dtype) )
    adds_res_pnp_ero = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = pnp_h_ero[None].type( target.dtype) )
       
    
    adds_init = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = h_real_est[None].type( target.dtype))
    
    # Append results to list
    desig_ls.append(int( unique_desig[1]) )
    adds_init_ls.append(float(adds_init.detach()))
    adds_gt_ls.append(float(adds_res_gt_flow.detach()))
    adds_erode_ls.append(float(adds_res_gt_flow_eroded.detach()))    
    adds_n1_ls.append(float(adds_res_n1.detach()))
    adds_n2_ls.append(float(adds_res_n2.detach()))
    adds_pnp_gt_ls.append(float(adds_res_pnp_gt.detach()))
    adds_pnp_ero_ls.append(float(adds_res_pnp_ero.detach()))

    if adds_gt_ls[-1] > 0.02 :
        print(f'Failed Iteration {j} > 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
        
        no = torch.norm(P_real_trafo- P_ren_in_center, dim=1)
        print( f'    shape of use points: ',P_real_trafo.shape )
        print( f'    number of point with a distance greater 0.2 {torch.sum( no>0.2 )} ' )
        print( f'    number of point with a distance greater 0.02 {torch.sum( no>0.02 )} ' )
        print( f'    number of point with a distance smaller 0.02 {torch.sum( no<0.02 )} ' )
        
        
        plot1 = True
    else:
        print(f'Suc Iteration {j} < 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
        plot1 = False
    if idxmax != -1:
        plot1 = True
        
    

    if plot1:
        continue
#         print("Real depth map cropped")
#         real_depth_img = Drawer().disp_img_1d(real_d[b].numpy(),ret=True)
#         real_depth_img = np.repeat( real_depth_img[:,:,None],3, axis=2)
#         print("Render depth map cropped")
#         render_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#         render_depth_img = np.repeat( render_depth_img[:,:,None],3, axis=2)
#         Drawer().disp_img_1d(flow_mask[0],ret=True)

#         sub = max(1,int( P_real_in_center.shape[0]/100 ) )
#         plot_two_pcd_line(P_real_in_center[::sub].numpy(), P_ren_in_center[::sub].numpy() )
#         plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )

#         print(f"Real Image, Estimated Points given GT Flow {P_real_in_center.shape}")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')
#         print("Real Image Cropped, Estimated Points given GT Flow")
#         visualizer.plot_estimated_pose_on_bb( tag = f"_",
#                         epoch = 1,
#                         img= real_depth_img,
#                         tl = real_tl[0],
#                         br = real_br[0],
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')

#         fil = label == unique_desig[1]    
#         real_img_original[b][ fil[0][:,:,None].repeat(1,1,3) ] = 255
#         print("Real Image, Estimated Points given GT Flow with label is white")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')

#         print("Render Image, Rendered Points not transformed reprojected")
#         visualizer.plot_estimated_pose( tag = f"_",
#                             epoch = 1,
#                             img= render_img_original[b].cpu().numpy(),
#                             points = copy.deepcopy(P_ren_in_center.cpu().numpy()),
#                             store = False,
#                             jupyter=True,
#                             K = K_ren.cpu().numpy(),
#                             H = np.eye(4),
#                             method='def')
#         print("Render Image Cropped, Rendered Points not transformed reprojected")
#         visualizer.plot_estimated_pose_on_bb( tag = f"_",
#                             epoch = 1,
#                             img= render_depth_img,
#                             points = copy.deepcopy(P_ren_in_center.cpu().numpy()),
#                             tl = ren_tl[0],
#                             br = ren_br[0],
#                             store = False,
#                             jupyter=True,
#                             K = K_ren.cpu().numpy(),
#                             H = np.eye(4),
#                             method='def')

        print('Valid Flow Mask')
        visualizer.plot_segmentation('tag', 1, flow_mask_eroded[b] , store=False, method='def', jupyter=True)
        visualizer.plot_segmentation('tag', 1, flow_mask[b]  , store=False, method='def', jupyter=True)

        print("Corrospondence")
        visualizer.plot_corrospondence(tag=f'_',
                                           epoch=0,
                                            u_map=u_map[0], 
                                            v_map=v_map[0], 
                                            flow_mask=flow_mask[0], 
                                            real_img=real_img[0], 
                                            render_img=render_img[0],
                                            store=False,
                                            jupyter=True,
                                            coloful=True,
                                            method='def',
                                            res_h=1,
                                            res_w=1)

        
# print(f'Result, GT Avg', sum(adds_gt_ls)/len(adds_gt_ls), 'Errode Avg', sum(adds_erode_ls)/len(adds_erode_ls) )


START


/home/jonfrey/PLR3/src/loaders_v2/dataset_ycb.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  init_trans = torch.normal(mean=torch.tensor(gt_trans), std=nt)
/home/jonfrey/PLR3/src/loaders_v2/dataset_ycb.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  init_trans = torch.normal(mean=torch.tensor(gt_trans), std=nt)
/home/jonfrey/miniconda3/envs/track_latest/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for det

Processed 0/1000


/home/jonfrey/miniconda3/envs/track_latest/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Suc Iteration 0 < 0.02 ADD-S, GT 0.010037760250270367 ERRODED 0.009412761777639389

Failed Iteration 1 > 0.02 ADD-S, GT 0.028129657730460167 ERRODED 0.035541702061891556
    shape of use points:  torch.Size([1000, 3])
    number of point with a distance greater 0.2 0 
    number of point with a distance greater 0.02 131 
    number of point with a distance smaller 0.02 869 

Suc Iteration 2 < 0.02 ADD-S, GT 0.0020052525214850903 ERRODED 0.0021888420451432467

Suc Iteration 3 < 0.02 ADD-S, GT 0.007318316027522087 ERRODED 0.007879034616053104

Failed Iteration 4 > 0.02 ADD-S, GT 0.024212436750531197 ERRODED 0.025562791153788567
    shape of use points:  torch.Size([1000, 3])
    number of point with a distance greater 0.2 0 
    number of point with a distance greater 0.02 0 
    number of point with a distance smaller 0.02 1000 

Failed Iteration 5 > 0.02 ADD-S, GT 0.041077621281147 ERRODED 0.038842447102069855
    shape of use points:  torch.Size([1000, 3])
    number of point with a 

...


Suc Iteration 648 < 0.02 ADD-S, GT 0.01707473397254944 ERRODED 0.013824782334268093

Failed Iteration 649 > 0.02 ADD-S, GT 0.10380379110574722 ERRODED 0.10994094610214233
    shape of use points:  torch.Size([1000, 3])
    number of point with a distance greater 0.2 0 
    number of point with a distance greater 0.02 18 
    number of point with a distance smaller 0.02 982 
Processed 650/1000

Suc Iteration 650 < 0.02 ADD-S, GT 0.019231200218200684 ERRODED 0.019813083112239838

Suc Iteration 651 < 0.02 ADD-S, GT 0.007028513588011265 ERRODED 0.006949001457542181

Suc Iteration 652 < 0.02 ADD-S, GT 0.00790268275886774 ERRODED 0.008177836425602436

Suc Iteration 653 < 0.02 ADD-S, GT 0.003843681188300252 ERRODED 0.0039651659317314625

Suc Iteration 654 < 0.02 ADD-S, GT 0.015285181812942028 ERRODED 0.015164774842560291

Suc Iteration 655 < 0.02 ADD-S, GT 0.0042432197369635105 ERRODED 0.004444735590368509

Suc Iteration 656 < 0.02 ADD-S, GT 0.007117102853953838 ERRODED 0.0069039263762533665


Suc Iteration 720 < 0.02 ADD-S, GT 0.006357629783451557 ERRODED 0.005937513895332813

Suc Iteration 721 < 0.02 ADD-S, GT 0.0055772545747458935 ERRODED 0.005244272295385599

Suc Iteration 722 < 0.02 ADD-S, GT 0.011822515167295933 ERRODED 0.01023554801940918

Suc Iteration 723 < 0.02 ADD-S, GT 0.007293317932635546 ERRODED 0.007901197299361229

Suc Iteration 724 < 0.02 ADD-S, GT 0.011366119608283043 ERRODED 0.01169809140264988

Suc Iteration 725 < 0.02 ADD-S, GT 0.008459934964776039 ERRODED 0.00713719055056572

Suc Iteration 726 < 0.02 ADD-S, GT 0.013855718076229095 ERRODED 0.015576275065541267

Suc Iteration 727 < 0.02 ADD-S, GT 0.008611458353698254 ERRODED 0.00854002870619297

Suc Iteration 728 < 0.02 ADD-S, GT 0.011184824630618095 ERRODED 0.012122618034482002

Suc Iteration 729 < 0.02 ADD-S, GT 0.007495049852877855 ERRODED 0.007844720035791397
Processed 730/1000

Suc Iteration 730 < 0.02 ADD-S, GT 0.012374160811305046 ERRODED 0.011831140145659447

Failed Iteration 731 > 0.02 ADD-S, GT


Failed Iteration 793 > 0.02 ADD-S, GT 0.020741993561387062 ERRODED 0.02188052609562874
    shape of use points:  torch.Size([1000, 3])
    number of point with a distance greater 0.2 0 
    number of point with a distance greater 0.02 5 
    number of point with a distance smaller 0.02 995 

Suc Iteration 794 < 0.02 ADD-S, GT 0.015280010178685188 ERRODED 0.015140650793910027

Suc Iteration 795 < 0.02 ADD-S, GT 0.003407508134841919 ERRODED 0.003567200619727373

Suc Iteration 796 < 0.02 ADD-S, GT 0.008402802050113678 ERRODED 0.009553163312375546

Suc Iteration 797 < 0.02 ADD-S, GT 0.008826952427625656 ERRODED 0.008572803810238838

Suc Iteration 798 < 0.02 ADD-S, GT 0.005513169802725315 ERRODED 0.005834921728819609

Suc Iteration 799 < 0.02 ADD-S, GT 0.002144402591511607 ERRODED 0.0024842298589646816
Processed 800/1000

Suc Iteration 800 < 0.02 ADD-S, GT 0.010156038217246532 ERRODED 0.007001239340752363

Failed Iteration 801 > 0.02 ADD-S, GT 0.02253664843738079 ERRODED 0.0258642360568046


Suc Iteration 863 < 0.02 ADD-S, GT 0.003633848624303937 ERRODED 0.003613437991589308

Suc Iteration 864 < 0.02 ADD-S, GT 0.006057023070752621 ERRODED 0.005534814670681953

Suc Iteration 865 < 0.02 ADD-S, GT 0.002597692422568798 ERRODED 0.0021522140596061945

Suc Iteration 866 < 0.02 ADD-S, GT 0.008609695360064507 ERRODED 0.006203681696206331

Suc Iteration 867 < 0.02 ADD-S, GT 0.0033562136813998222 ERRODED 0.0034698862582445145

Suc Iteration 868 < 0.02 ADD-S, GT 0.0033856553491204977 ERRODED 0.0034910691902041435

Suc Iteration 869 < 0.02 ADD-S, GT 0.01824229024350643 ERRODED 0.01842682622373104
Processed 870/1000

Suc Iteration 870 < 0.02 ADD-S, GT 0.016349531710147858 ERRODED 0.017394570633769035

Suc Iteration 871 < 0.02 ADD-S, GT 0.0014389039715752006 ERRODED 0.0017644090112298727

Failed Iteration 872 > 0.02 ADD-S, GT 0.03997866436839104 ERRODED 0.040142618119716644
    shape of use points:  torch.Size([1000, 3])
    number of point with a distance greater 0.2 0 
    number of p


Suc Iteration 940 < 0.02 ADD-S, GT 0.0030449938494712114 ERRODED 0.0030633914284408092

Suc Iteration 941 < 0.02 ADD-S, GT 0.00610872870311141 ERRODED 0.005671141669154167

Suc Iteration 942 < 0.02 ADD-S, GT 0.01144254207611084 ERRODED 0.011596374213695526

Suc Iteration 943 < 0.02 ADD-S, GT 0.013744823634624481 ERRODED 0.015919381752610207

Suc Iteration 944 < 0.02 ADD-S, GT 0.0013830590760335326 ERRODED 0.002570410957559943

Suc Iteration 945 < 0.02 ADD-S, GT 0.0017242601606994867 ERRODED 0.0019832944963127375

Suc Iteration 946 < 0.02 ADD-S, GT 0.0024419748224318027 ERRODED 0.0022654198110103607

Suc Iteration 947 < 0.02 ADD-S, GT 0.004689300432801247 ERRODED 0.004729022271931171

Suc Iteration 948 < 0.02 ADD-S, GT 0.018952706828713417 ERRODED 0.01954824849963188

Suc Iteration 949 < 0.02 ADD-S, GT 0.00821351632475853 ERRODED 0.007882243022322655
Processed 950/1000

Suc Iteration 950 < 0.02 ADD-S, GT 0.004656362812966108 ERRODED 0.00869059655815363

Suc Iteration 951 < 0.02 ADD-S, 

In [31]:
import pandas

data = np.array( [desig_ls,adds_gt_ls,adds_init_ls,adds_erode_ls, adds_n1_ls, adds_n2_ls, adds_pnp_gt_ls, adds_pnp_ero_ls] ).T
df = pandas.DataFrame( data, columns = ['ID','ADD-S GT','ADD-S INITAL','ADD-S Eroded Mask', 'ADD-S Random', 'ADD-S Bias', 'ADD-S PNP GT', 'ADD-S PNP Ero'] )
df

,ID,ADD-S GT,ADD-S INITAL,ADD-S Eroded Mask,ADD-S Random,ADD-S Bias,ADD-S PNP GT,ADD-S PNP Ero
0,14.0,0.010038,0.031212,0.009413,0.009606,0.010907,0.021060,0.021487
1,17.0,0.028130,0.040551,0.035542,0.035904,0.035704,0.062383,0.271818
2,19.0,0.002005,0.010307,0.002189,0.001960,0.003808,0.003635,0.003703
3,4.0,0.007318,0.024532,0.007879,0.007149,0.011192,0.010879,0.010573
4,18.0,0.024212,0.035395,0.025563,0.025608,0.025781,0.062177,0.443799
...,...,...,...,...,...,...,...,...
969,4.0,0.005544,0.044807,0.005518,0.005574,0.005571,0.025860,0.024910
970,21.0,0.004421,0.033833,0.004435,0.004629,0.004843,0.005435,0.006220
971,10.0,0.029724,0.046952,0.030057,0.029920,0.030642,0.037100,0.035370
972,3.0,0.002448,0.022561,0.002407,0.002188,0.005499,0.003811,0.006725


In [32]:
print('Random: Takeing Ground Truth Flow and adding noise uniform in between +20 -20 pixels for each pixel')
print('Bias: Takeing Ground Truth Flow and offsetting the flow by a random value in between +20 -20 pixels for the full image')
df.groupby(['ID']).mean()

Random: Takeing Ground Truth Flow and adding noise uniform in between +20 -20 pixels for each pixel
Bias: Takeing Ground Truth Flow and offsetting the flow by a random value in between +20 -20 pixels for the full image


,ADD-S GT,ADD-S INITAL,ADD-S Eroded Mask,ADD-S Random,ADD-S Bias,ADD-S PNP GT,ADD-S PNP Ero
ID,,,,,,,
1.0,0.005648,0.036748,0.005405,0.005237,0.009325,0.008285,0.010191
2.0,0.007635,0.041665,0.006524,0.006209,0.015146,0.008780,0.009570
3.0,0.010521,0.037349,0.010563,0.011409,0.028394,0.026940,0.028221
4.0,0.011626,0.035669,0.011681,0.010973,0.013982,0.026593,0.030557
5.0,0.002916,0.036959,0.003035,0.003488,0.047049,0.005380,0.005848
6.0,0.033227,0.033875,0.031424,0.030795,0.033336,1.457534,0.181489
7.0,0.009279,0.037736,0.009998,0.009926,0.012715,0.037107,0.036274
8.0,0.017378,0.032137,0.017416,0.017519,0.017244,0.073932,0.071005
9.0,0.022048,0.032810,0.020886,0.020807,0.024020,0.026202,0.046964


In [ ]:
print("MEAN:\n", df.mean() )
print("\n \nSTD:\n", df.std() )
print("\n \nMAX:\n", df.max())
print("\n \nIDXMAX:\n", df.idxmax())

idxmax = df.idxmax()[0]
print(f'\n \nWorst Sample Index is {idxmax}')

In [ ]:
ls = adds_erode_ls
n_bins = 100
failures_filter = 0.1
fig, axs = plt.subplots(3, 1, sharey=True, tight_layout=True)
axs[0].hist(adds_init_ls, bins=n_bins)

adds_gt_f = [a for a in adds_gt_ls if a < failures_filter]
adds_erode_f = [a for a in adds_erode_ls if a < failures_filter]

axs[1].hist(adds_gt_f, bins=n_bins)
axs[2].hist(adds_erode_f, bins=n_bins)
fig.show()
